In [6]:
import os
# define the function to update particle id
def updateParticleId(idparticle, idlist):
    #if length of idlist is 1(scattered), then pop the corresponding id from idparticle
    if len(idlist) == 1:
        idparticle.pop(idlist[0]-1)
        return idparticle
    #if length of idlist is 2(collided), then move the last id in idparticle to the position of the larger id in idlist
    elif len(idlist) == 2:
        idparticle.pop(max(idlist)-1)
        return idparticle
    #else throw an error
    else:
        raise ValueError('The length of idlist should be either 1(scattered) or 2(collided)')
    
# define the function to create a directory if it does not exist
def createDir(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return

# read events(collisions and scatters) and sort them by time
# input: meta_fileinfo: a list of tuples, each tuple contains the identifier of the event file('col' or 'sca'), the file name and the column name dictionary
def readEvents(meta_fileinfo):
    event_list = []
    for identifier, fname, colname in meta_fileinfo: # read events file one by one
        with open(fname, 'r') as f:
            for line in f:
                params = line.split()
                # catch index out of range error
                try:
                    t = float(params[colname['t']])
                    if identifier == 'col': # identified as collision event data
                        id1 = int(params[colname['id1']])
                        id2 = int(params[colname['id2']])
                        event_list.append([t, (id1, id2), identifier])
                    elif identifier == 'sca': # identified as scatter event data
                        id = int(params[colname['id']])
                        event_list.append([t, (id,), identifier])
                    else:
                        raise ValueError('identifier should be either col or sca')
                except IndexError:
                    print("IndexError: ", line, " in ", fname, " is not a valid event record. Passed.")
                    continue
    # sort the event list by time
    event_list.sort(key = lambda x: x[0])
    event_list.append([float('inf'), (), 'terminate']) # add a dummy event at the end of the list
    return event_list

# read orbit records and group them by particle id
def groupById(event_list, fname, colname, output_dir=''):
    # create output directory if it does not exist
    if output_dir is not '':
        createDir(output_dir)
    # read orbit records
    with open(fname, 'r') as f:
        # initialize particle_list, init flag, event_id and particle_pos
        particle_list = [] # particle_list is a list of particle id in the order of orbit records
        particle_list_not_initialized = True
        event_id = 0 # the id of the current event
        particle_pos = 0 # the position of the current particle in the particle list
        # read orbit records one by one
        for line in f:
            # read an orbit record
            params_orbit = line.split()
            # catch index out of range error
            try:
                # extract t as a scientific number
                t = float(params_orbit[colname['t']])
                # extract np as an integer
                np = int(params_orbit[colname['#p']]) - 1
            except IndexError:
                print("IndexError: ", line, " in ", fname, " is not a valid orbit record. Passed.")
                continue
            # initialize the particle list at the first orbit record
            if particle_list_not_initialized:
                particle_list = [i+1 for i in range(np)]
                particle_list_not_initialized = False
                print("event_id: ", event_id, "\tt:", t, "\t#p:", np, "\ttype:", event_list[event_id][2], "\tparticle_list: ", particle_list)
            # update particle id
            while t > event_list[event_id][0]:
                particle_list = updateParticleId(particle_list, event_list[event_id][1])
                # reset particle_pos
                particle_pos = 0
                # wait for the next event
                event_id += 1
                # show current status
                print("event_id: ", event_id, "\tt:", t, "\t#p:", np, "\ttype:", event_list[event_id][2], "\tparticle_list: ", particle_list)
            # group records by particle id
            with open(output_dir + "particle_" + str(particle_list[particle_pos]) + ".txt", 'a') as fw:
                fw.write(line)
            # update particle_pos
            particle_pos += 1
            particle_pos %= len(particle_list)
    return
    

<>:53: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:53: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/m6/xzr56v3s0rx1lt1w2h7sq5p80000gn/T/ipykernel_3753/3075565417.py:53: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if output_dir is not '':


In [7]:
# data file info
# output file dir
output_dir = './groupById/'
# file name
fname_orb = 'orbit-m.txt'
fname_col = 'collision.txt'
fname_sca = 'scatter.txt'
# column name
colname_orb = {'t':0, '#p':1}
colname_col = {'t':0, 'id1':1, 'id2':9}
colname_sca = {'t':0, 'id':1}
# create event list
event_list = readEvents([ # read events data from following files one by one
    ('col', fname_col, colname_col),
    ('sca', fname_sca, colname_sca)
])
# read orbit records and group them by particle id
groupById(event_list, fname_orb, colname_orb, output_dir)

event_id:  0 	t: 0.0 	#p: 23 	type: col 	particle_list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
event_id:  1 	t: 420000.0 	#p: 21 	type: col 	particle_list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23]
event_id:  2 	t: 420000.0 	#p: 21 	type: sca 	particle_list:  [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23]
event_id:  3 	t: 580000.0 	#p: 20 	type: col 	particle_list:  [1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23]
event_id:  4 	t: 790000.0 	#p: 19 	type: col 	particle_list:  [1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23]
event_id:  5 	t: 2520000.0 	#p: 18 	type: col 	particle_list:  [1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 15, 16, 17, 18, 19, 21, 22, 23]
event_id:  6 	t: 2960000.0 	#p: 17 	type: col 	particle_list:  [1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 15, 16, 17, 18, 21, 22, 23]
event_id:  7 	t: 3270000.0 	#p: 16 	type: col 	particle_list